In [ ]:
import faiss
import finnhub
import os
import numpy as np
import pandas as pd
import requests
import openai
import semantic_kernel as sk
import torch
from transformers import pipeline
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from alpaca.data import StockHistoricalDataClient
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
FINNHUB_API_KEY = os.environ['FINNHUB_API_KEY']
ALPACA_API_KEY = os.environ['ALPACA_KEY']
ALPACA_SECRET_KEY = os.environ['ALPACA_SECRET']
ALPACA_PAPER_API_KEY = os.environ['ALPACA_PAPER_API_KEY']
ALPACA_PAPER_SECRET_KEY = os.environ['ALPACA_PAPER_API_SECRET']
BASE_URL = 'https://paper-api.alpaca.markets'

In [ ]:
# setup language models
gpt4_kernel = sk.Kernel()
gpt4_kernel.add_service(
    OpenAIChatCompletion(
        service_id="gpt4o",
        ai_model_id="gpt-4o",
        api_key=OPENAI_API_KEY
    ),
)
gpt3_kernel = sk.Kernel()
gpt3_kernel.add_service(
    OpenAIChatCompletion(
        service_id="gpt3.5-turbo",
        ai_model_id="gpt-3.5-turbo",
        api_key=OPENAI_API_KEY
    ),
)

# examples of adding functions to the kernels
gpt4_kernel.add_function(function_name="get_stock_price", plugin_name="stock_info_plugin", prompt="What is the stock price of {symbol}?")
gpt4_kernel.add_function(function_name="get_company_profile", plugin_name="stock_info_plugin", prompt="Tell me about the company {symbol}.")
gpt4_kernel.add_function(function_name="get_historical_data", plugin_name="stock_info_plugin", prompt="Give me the historical data for {symbol}.")
gpt3_kernel.add_function(function_name="get_stock_price", plugin_name="stock_info_plugin", prompt="What is the stock price of {symbol}?")
gpt3_kernel.add_function(function_name="get_company_profile", plugin_name="stock_info_plugin", prompt="Tell me about the company {symbol}.")
gpt3_kernel.add_function(function_name="get_historical_data", plugin_name="stock_info_plugin", prompt="Give me the historical data for {symbol}.")


# for using Phi-3 locally
qa_pipeline = pipeline('text-generation', model='microsoft/Phi-3-mini-128k-instruct')

In [ ]:
test_function = gpt3_kernel.get_function(function_name="get_stock_price", plugin_name="stock_info_plugin")
request_settings = {
    "prompt": "What is the stock price of AAPL?",
    "max_tokens": 150,  # Adjust as needed
    "temperature": 0.7,
    "top_p": 0.9
}

# Invoke the function
result = await gpt3_kernel.invoke(test_function, **request_settings)
print(result.get_inner_content().to_dict()['choices'][0]['message']['content'])

In [ ]:
finnhub_client = finnhub.Client(FINNHUB_API_KEY)
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True)

# Initialize FAISS index
d = 512  # Example dimensionality
index = faiss.IndexIVFPQ(faiss.IndexFlatL2(d), d, 256, 8, 8)
symbol_mapping = {}

def fetch_stock_symbols():
    #TODO: Fetch stock symbols from API
    return ["AAPL", "MSFT", "AMZN", "GOOGL", "FB", "JPM", "JNJ", "NVDA", "PG", "DIS"]

def fetch_stock_data(symbol):
    # Example function to fetch stock data
    url = f"https://api.example.com/stock/{symbol}/fundamentals"
    response = requests.get(url)
    data = response.json()
    return data

def preprocess_data(data):
    # Convert your data to a vector
    # Ensure the order of features matches the dimensionality 'd'
    features = ['open', 'high', 'low', 'close', 'volume']
    vector = np.array([data[feature] for feature in features]).astype('float32')
    return vector

def store_data_in_faiss(symbol, data):
    vector = preprocess_data(data)
    index.add(np.array([vector]))
    symbol_mapping[index.ntotal - 1] = symbol

def fetch_latest_data_from_faiss():
    # Example function to fetch latest data from FAISS
    k = 1  # Number of nearest neighbors
    query_vector = np.random.random(d).astype('float32')  # Example query vector
    D, I = index.search(np.array([query_vector]), k)
    symbols = [symbol_mapping[i] for i in I[0]]
    return symbols

def create_debate_prompt(stock_data):
    prompt = [
        {"role": "system", "content": "You are a financial expert."},
        {"role": "user", "content": f"Analyze the following stock data and debate: {stock_data}"}
    ]
    return prompt

def debate_agents(stock_data):
    prompt = create_debate_prompt(stock_data)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=prompt,
        temperature=0.7
    )
    return parse_debate_response(response)

def parse_debate_response(response):
    decisions = [message['content'] for message in response['choices']]
    decision_counts = {"buy": 0, "sell": 0, "hold": 0}
    for decision in decisions:
        if "buy" in decision:
            decision_counts["buy"] += 1
        elif "sell" in decision:
            decision_counts["sell"] += 1
        else:
            decision_counts["hold"] += 1
    return max(decision_counts, key=decision_counts.get)

def execute_trade(decision, symbol):
    if decision == 'buy':
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC
        )
    elif decision == 'sell':
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.GTC
        )
    
    trading_client.submit_order(order)

In [ ]:
# List of stock symbols to evaluate
stocks = ['AAPL', 'MSFT', 'GOOGL']

def get_stock_fundamentals(stock_symbol):
    """Fetches basic fundamental data for a given stock symbol using Finnhub API."""
    try:
        # Get basic financials
        financials = finnhub_client.company_basic_financials(stock_symbol, 'all')

        # Extract relevant data
        metric = financials['metric']
        fundamentals = {
            'symbol': stock_symbol,
            'peRatioTTM': metric.get('peNormalizedAnnual', 'N/A'),
            'pbRatio': metric.get('pbAnnual', 'N/A'),
            'dividendYieldTTM': metric.get('currentDividendYieldTTM', 'N/A'),
            'epsTTM': metric.get('epsTTM', 'N/A'),
            'marketCapitalization': metric.get('marketCapitalization', 'N/A'),
            '52WeekHigh': metric.get('52WeekHigh', 'N/A'),
            '52WeekLow': metric.get('52WeekLow', 'N/A'),
        }
        return fundamentals
    except Exception as e:
        print(f"Error fetching data for {stock_symbol}: {e}")
        return None

def evaluate_stocks(stock_list):
    """Evaluates a list of stocks and prints their fundamental data."""
    results = []
    for stock in stock_list:
        fundamentals = get_stock_fundamentals(stock)
        if fundamentals:
            results.append(fundamentals)
    
    return results

import finnhub
import os

# List of stock symbols to evaluate
stocks = ['AAPL', 'MSFT', 'GOOGL']

def get_stock_fundamentals(stock_symbol):
    """Fetches basic fundamental data for a given stock symbol using Finnhub API."""
    try:
        # Get basic financials
        financials = finnhub_client.company_basic_financials(stock_symbol, 'all')

        # Get current price
        quote = finnhub_client.quote(stock_symbol)
        current_price = quote['c']

        # Extract relevant data
        metric = financials['metric']
        fundamentals = {
            'symbol': stock_symbol,
            'current_price': current_price,
            '10DayAverageTradingVolume': metric.get('10DayAverageTradingVolume', 'N/A'),
            '13WeekPriceReturnDaily': metric.get('13WeekPriceReturnDaily', 'N/A'),
            '26WeekPriceReturnDaily': metric.get('26WeekPriceReturnDaily', 'N/A'),
            '52WeekHigh': metric.get('52WeekHigh', 'N/A'),
            '52WeekLow': metric.get('52WeekLow', 'N/A'),
            '52WeekPriceReturnDaily': metric.get('52WeekPriceReturnDaily', 'N/A'),
            '5DayPriceReturnDaily': metric.get('5DayPriceReturnDaily', 'N/A'),
            'peRatioTTM': metric.get('peNormalizedAnnual', 'N/A'),
            'pbRatio': metric.get('pbAnnual', 'N/A'),
            'dividendYieldTTM': metric.get('currentDividendYieldTTM', 'N/A'),
            'epsTTM': metric.get('epsTTM', 'N/A'),
            'marketCapitalization': metric.get('marketCapitalization', 'N/A'),
            'currentRatioAnnual': metric.get('currentRatioAnnual', 'N/A'),
            'quickRatioAnnual': metric.get('quickRatioAnnual', 'N/A'),
            'debt/equityAnnual': metric.get('totalDebt/totalEquityAnnual', 'N/A'),
            'grossMarginAnnual': metric.get('grossMarginAnnual', 'N/A'),
            'netProfitMarginAnnual': metric.get('netProfitMarginAnnual', 'N/A'),
            'operatingMarginAnnual': metric.get('operatingMarginAnnual', 'N/A'),
            'revenueGrowth3Y': metric.get('revenueGrowth3Y', 'N/A'),
            'revenueGrowth5Y': metric.get('revenueGrowth5Y', 'N/A'),
            'revenueGrowthQuarterlyYoy': metric.get('revenueGrowthQuarterlyYoy', 'N/A'),
            'epsGrowth3Y': metric.get('epsGrowth3Y', 'N/A'),
            'epsGrowth5Y': metric.get('epsGrowth5Y', 'N/A'),
            'bookValuePerShareAnnual': metric.get('bookValuePerShareAnnual', 'N/A'),
            'cashFlowPerShareTTM': metric.get('cashFlowPerShareTTM', 'N/A'),
            'enterpriseValue': metric.get('enterpriseValue', 'N/A'),
        }
        return fundamentals
    except Exception as e:
        print(f"Error fetching data for {stock_symbol}: {e}")
        return None

def evaluate_stocks(stock_list):
    """Evaluates a list of stocks and prints their fundamental data."""
    results = []
    for stock in stock_list:
        fundamentals = get_stock_fundamentals(stock)
        if fundamentals:
            results.append(fundamentals)
    
    return results

if __name__ == "__main__":
    stock_fundamentals = evaluate_stocks(stocks)
    for stock in stock_fundamentals:
        print(stock)

print(pd.DataFrame(stock_fundamentals))

In [ ]:
import pandas as pd
pd.DataFrame(finnhub_client.company_basic_financials('MSFT', 'All')).T.to_json()

In [ ]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest

trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=False)

# Get our account information.
account = trading_client.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

In [ ]:
trading_client.get_all_positions()

In [ ]:
symbol = "MSFT"  # replace with your symbol
url = f"https://data.alpaca.markets/v2/stocks/{symbol}/quotes/latest"

headers = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY
}

response = requests.get(url, headers=headers)
data = response.json()
print(data)

In [ ]:
url = "https://data.alpaca.markets/v2/positions"
headers = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY
}

response = requests.get(url, headers=headers)
data = response.json()
print(data)